<a href="https://colab.research.google.com/github/treyra/CS155_miniproject/blob/data_visualization_v1/nn_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.config.experimental.list_physical_devices('GPU')

SystemError: ignored

In [6]:
df_train = pd.read_csv('https://raw.githubusercontent.com/treyra/CS155_miniproject/data_visualization_v1/data_cleanup/traindata_imputed_normalized_standard.csv',sep=',', header=0,index_col=0)
df_train.head()

,id,LATITUDE,LONGITUDE,STATE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,FIPS_CODE,SOURCE_REPORTING_UNIT_NAME,YEAR,DAY,LABEL,0,1,2,3
0,0,0.988292,-0.985738,1,0.054167,-0.026526,1.048773,-1.143120,-0.871063,0.0,0.002732,1,1.0,0.0,0.0,0.0
1,1,-0.401537,0.925104,-1,0.464583,-0.025770,-1.922012,0.386764,-0.852154,0.0,0.002732,4,0.0,0.0,0.0,1.0
2,2,-0.911691,1.066427,-1,0.666667,-0.026547,-1.906459,1.140289,-0.852154,0.0,0.002732,2,0.0,1.0,0.0,0.0
3,3,-0.814128,1.109280,-1,0.506250,-0.023489,-1.890905,-0.891945,-0.852154,0.0,0.002732,4,0.0,0.0,0.0,1.0
4,4,-0.448755,-0.759942,1,0.549583,-0.026456,0.675480,-0.503766,-0.833245,0.0,0.002732,2,0.0,1.0,0.0,0.0


In [7]:
df_test = pd.read_csv('https://raw.githubusercontent.com/treyra/CS155_miniproject/data_visualization_v1/data_cleanup/testdata_imputed_normalized_standard.csv',sep=',', header=0,index_col=0)
df_test.head()

,id,LATITUDE,LONGITUDE,STATE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,FIPS_CODE,SOURCE_REPORTING_UNIT_NAME,YEAR,DAY
0,285382,-0.232600,-0.828839,1,0.66875,-0.026456,0.768803,-0.435263,-0.303789,1.058824,0.002732
1,285383,-0.335939,-0.760377,1,0.64375,-0.026526,0.784357,-0.435263,0.320212,1.058824,0.002732
2,285384,0.945131,-0.982574,1,0.50000,-0.026526,0.628819,-0.001415,-0.871063,1.058824,0.002732
3,285385,-0.817080,0.998962,-1,0.87500,-0.026314,-1.579828,0.500935,2.419125,1.058824,0.005464
4,285386,-0.682598,-0.695973,1,0.05250,-0.026526,1.204311,-0.960447,-0.681972,1.058824,0.005464


In [8]:
# drop features
df_test = df_test.drop(['FIPS_CODE', 'YEAR', 'SOURCE_REPORTING_UNIT_NAME','STATE'],axis=1)
df_train = df_train.drop(['FIPS_CODE', 'YEAR', 'SOURCE_REPORTING_UNIT_NAME','STATE'],axis=1)
df_test.head()

,id,LATITUDE,LONGITUDE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,DAY
0,285382,-0.232600,-0.828839,0.66875,-0.026456,0.768803,0.002732
1,285383,-0.335939,-0.760377,0.64375,-0.026526,0.784357,0.002732
2,285384,0.945131,-0.982574,0.50000,-0.026526,0.628819,0.002732
3,285385,-0.817080,0.998962,0.87500,-0.026314,-1.579828,0.005464
4,285386,-0.682598,-0.695973,0.05250,-0.026526,1.204311,0.005464


In [9]:
# make the data ready for training
# load dataset
df_train_X = df_train.drop(['id','LABEL','0','1','2','3'], axis =1)
df_train_y = df_train.drop(['id','LATITUDE','LONGITUDE','DISCOVERY_TIME','FIRE_SIZE','FIPS_NAME','DAY','LABEL'], axis=1)

X = df_train_X.to_numpy()
y = df_train_y.to_numpy()
n_inputs, n_outputs = X.shape[1], y.shape[1]
print(X)
print(y)

[[ 0.98829209 -0.98573763  0.05416667 -0.02652628  1.04877251  0.00273224]
 [-0.40153698  0.92510448  0.46458333 -0.02577049 -1.92201243  0.00273224]
 [-0.91169054  1.06642697  0.66666667 -0.02654747 -1.90645859  0.00273224]
 ...
 [-0.72827585  1.05320738  0.8875     -0.02658985 -0.33551995  0.99453552]
 [-0.34713874 -0.80024373  0.54958333 -0.02638501  0.67548016  0.99453552]
 [-0.51010212 -0.80233015  0.6        -0.02589057  0.67548016  0.99726776]]
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [10]:
import re
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
from keras import Sequential

In [11]:
def create_model(dense_layer_sizes = [32], optimizer="adam", dropout=0.1, init='he_uniform', dense_nparams=256):
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(6,), kernel_initializer=init,)) 
    model.add(Dropout(dropout), )
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size, activation='relu'))
        model.add(Dropout(dropout), )
    model.add(Dense(4, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=1)
# define the grid search parameters
# dense_layer_sizes = [(200,150,100),(250,100,50),(100,50,25),(60,30,10)]
# batch_sizes = [16,32,64]
# epochs = [10,40,50]
# optimizers = ['RMSprop', 'Adam', 'Adamax', 'sgd'],
# dropout = [0,0.1,0.2]
dense_layer_sizes = [(200,150,100),(250,100,25)]#,(50,25,15),(60,30,10)]
batch_sizes = [16]
epochs = [30]
optimizers = ['Adam']
dropout = [0]#,0.1]
#
param_grid = dict(dense_layer_sizes=dense_layer_sizes, epochs=epochs, batch_size=batch_sizes,optimizer=optimizers,dropout=dropout)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc_ovr', n_jobs=1, cv=2, verbose = 2,return_train_score=True)
grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# make a prediction for test data
df_test_noid = df_test.drop(['id'], axis=1)
X_test = df_test_noid.to_numpy()
print(X_test.shape)

y_test = grid.best_estimator_.predict_proba(X_test)

print('Predicted: %s' % y_test[0])
print(y_test)
# save as csv_file in sample submission format
# put it together in a dataframe and save
from google.colab import  drive

def save_csv(df, savestr):
  drive.mount("/drive", force_remount=True)
  df.to_csv('/drive/My Drive/CS155_miniproject_1/'+str(savestr)+'.csv')

df = pd.DataFrame(y_test)
predictions = pd.DataFrame(df_test['id'])
predictions['P1']=y_test[:,0]
predictions['P2']=y_test[:,1]
predictions['P3']=y_test[:,2]
predictions['P4']=y_test[:,3]
predictions = predictions.set_index('id')
predictions.head()
save_csv(predictions,'predictions_neural_net_standard_logfiresize_gridsearch_opt')

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] batch_size=16, dense_layer_sizes=(200, 150, 100), dropout=0, epochs=30, optimizer=Adam 
Epoch 1/30


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


8919/8919 [==============================] - 26s 3ms/step - loss: 0.4348 - accuracy: 0.5689
Epoch 2/30
8919/8919 [==============================] - 26s 3ms/step - loss: 0.4118 - accuracy: 0.5900
Epoch 3/30
8919/8919 [==============================] - 25s 3ms/step - loss: 0.4056 - accuracy: 0.5986
Epoch 4/30
8919/8919 [==============================] - 26s 3ms/step - loss: 0.4008 - accuracy: 0.6070
Epoch 5/30
8919/8919 [==============================] - 26s 3ms/step - loss: 0.3983 - accuracy: 0.6074
Epoch 6/30
8919/8919 [==============================] - 25s 3ms/step - loss: 0.3962 - accuracy: 0.6113
Epoch 7/30
8919/8919 [==============================] - 25s 3ms/step - loss: 0.3948 - accuracy: 0.6118
Epoch 8/30
8919/8919 [==============================] - 26s 3ms/step - loss: 0.3931 - accuracy: 0.6144
Epoch 9/30
8919/8919 [==============================] - 28s 3ms/step - loss: 0.3917 - accuracy: 0.6165
Epoch 10/30
8919/8919 [==============================] - 26s 3ms/step - loss: 0.3911

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


 151/8919 [..............................] - ETA: 8s

...

8919/8919 [==============================] - 9s 982us/step
[CV]  batch_size=16, dense_layer_sizes=(200, 150, 100), dropout=0, epochs=30, optimizer=Adam, total=13.2min
[CV] batch_size=16, dense_layer_sizes=(250, 100, 25), dropout=0, epochs=30, optimizer=Adam 
Epoch 1/30
8919/8919 [==============================] - 29s 3ms/step - loss: 0.4363 - accuracy: 0.5670
Epoch 2/30
8919/8919 [==============================] - 28s 3ms/step - loss: 0.4136 - accuracy: 0.5874
Epoch 3/30
8919/8919 [==============================] - 29s 3ms/step - loss: 0.4071 - accuracy: 0.5954
Epoch 4/30
8919/8919 [==============================] - 29s 3ms/step - loss: 0.4014 - accuracy: 0.6008
Epoch 5/30
8919/8919 [==============================] - 30s 3ms/step - loss: 0.3989 - accuracy: 0.6061
Epoch 6/30
8919/8919 [==============================] - 31s 3ms/step - loss: 0.3963 - accuracy: 0.6083
Epoch 7/30
8919/8919 [==============================] - 30s 3ms/step - loss: 0.3946 - accuracy: 0.6111
Epoch 8/30
8919/8919

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


  89/8919 [..............................] - ETA: 10s

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


8919/8919 [==============================] - 10s 1ms/step
[CV]  batch_size=16, dense_layer_sizes=(250, 100, 25), dropout=0, epochs=30, optimizer=Adam, total=15.1min
[CV] batch_size=16, dense_layer_sizes=(250, 100, 25), dropout=0, epochs=30, optimizer=Adam 
Epoch 1/30
8919/8919 [==============================] - 30s 3ms/step - loss: 0.4287 - accuracy: 0.6023
Epoch 2/30
8919/8919 [==============================] - 29s 3ms/step - loss: 0.4062 - accuracy: 0.6212
Epoch 3/30
8919/8919 [==============================] - 29s 3ms/step - loss: 0.4023 - accuracy: 0.6244
Epoch 4/30
8919/8919 [==============================] - 30s 3ms/step - loss: 0.3969 - accuracy: 0.6285
Epoch 5/30
8919/8919 [==============================] - 29s 3ms/step - loss: 0.3946 - accuracy: 0.6301
Epoch 6/30
8919/8919 [==============================] - 30s 3ms/step - loss: 0.3931 - accuracy: 0.6315
Epoch 7/30
8919/8919 [==============================] - 29s 3ms/step - loss: 0.3903 - accuracy: 0.6343
Epoch 8/30
8919/8919 [

In [2]:
y_test = grid.best_estimator_.predict_proba(X_test)

print('Predicted: %s' % y_test[0])
print(y_test)

NameError: ignored

In [63]:
# # get the model
# def get_model(n_inputs, n_outputs):
#   # 0.792 in kaggle with dropped features and 40 epochs
#   model = Sequential()
#   model.add(Dense(5, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
#   model.add(Dropout(0), )
#   #model.add(Dense(3, input_dim=5, kernel_initializer='he_uniform', activation='relu')) # tried selu also 
#   model.add(Dense(3, input_dim=3, kernel_initializer='he_uniform', activation='relu')) # tried selu also 
#   model.add(Dropout(0), )
#   # last layer fix for multi-classification probabilities
#   model.add(Dense(n_outputs, activation='sigmoid'))
#   model.add(Dropout(0), )
#   model.compile(loss='binary_crossentropy', optimizer='adam') # tried adamax also
#   return model

# # load dataset
# df_train_X = df_train.drop(['id','LABEL','0','1','2','3'], axis =1)
# df_train_y = df_train.drop(['id','LATITUDE','LONGITUDE','DISCOVERY_TIME','FIRE_SIZE','FIPS_NAME','DAY','LABEL'], axis=1)

# # X = df_train_X.to_numpy()
# # y = df_train_y.to_numpy()
# n_inputs, n_outputs = X.shape[1], y.shape[1]
# # get model

# model = get_model(n_inputs, n_outputs)
# # fit the model on all data
# with tf.device('/device:GPU:0'):
#   # simple early stopping
#   #es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
#   # mc = ModelCheckpoint('best_model.h5', monitor='loss', mode='max', verbose=1, save_best_only=True)
#   history = model.fit(X, y, verbose=1, epochs=1)
# # make a prediction for test data
# df_test_noid = df_test.drop(['id'], axis=1)
# X_test = df_test_noid.to_numpy()
# print(X_test.shape)

# y_test = model.predict(X_test)

# print('Predicted: %s' % y_test[0])
# print(y_test)

8919/8919 [==============================] - 19s 2ms/step - loss: 0.4810
(73035, 6)
Predicted: [0.08812517 0.45361853 0.07741149 0.30435774]
[[0.08812517 0.45361853 0.07741149 0.30435774]
 [0.09311597 0.45237437 0.08085793 0.30605784]
 [0.10830888 0.4489263  0.09116152 0.31080157]
 ...
 [0.07964306 0.45588884 0.07147143 0.30127126]
 [0.09664863 0.4515298  0.08327775 0.30721548]
 [0.14771537 0.44164786 0.11691857 0.32096875]]


In [60]:
# yhat_classes2 = grid.best_estimator_.predict(X_test)
# print(yhat_classes2)

 166/4565 [>.............................] - ETA: 4s

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


4565/4565 [==============================] - 5s 1ms/step
[1 1 1 ... 1 1 1]


In [55]:
# # make a prediction for test data
# df_test_noid = df_test.drop(['id'], axis=1)
# X_test = df_test_noid.to_numpy()
# print(X_test.shape)

# y_test = grid.best_estimator_.predict(X_test)

# print('Predicted: %s' % y_test[0])
# print(y_test)

# yhat_classes2 = np.argmax(grid.best_estimator_.predict(X_test), axis=1)
# print(yhat_classes2)

(73035, 6)
 160/4565 [>.............................] - ETA: 4s

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


4565/4565 [==============================] - 4s 972us/step
Predicted: 1
[1 1 1 ... 1 1 1]
4565/4565 [==============================] - 4s 970us/step


AxisError: ignored

In [26]:
# kears_estimator = KerasClassifier(build_fn=create_model, verbose=1)

In [27]:
# estimator = Pipeline([("kc", kears_estimator)])

In [28]:

# # define the grid search parameters
# param_grid = {
#     'kc__epochs': [10, 100, ],
#     'kc__dense_nparams': [32, 256, 512],
#     'kc__init': [ 'uniform', 'zeros', 'normal', ], 
#     'kc__batch_size':[2, 16, 32],
#     'kc__optimizer':['RMSprop', 'Adam', 'Adamax', 'sgd'],
#     'kc__dropout': [0.5, 0.4, 0.3, 0.2, 0.1, 0]
# }

In [29]:
# kfold_splits = 5
# grid = GridSearchCV(estimator=estimator,  
#                     n_jobs=-1, 
#                     verbose=1,
#                     return_train_score=True,
#                     cv=kfold_splits,  #StratifiedKFold(n_splits=kfold_splits, shuffle=True)
                    # param_grid=param_grid,)

In [30]:
# grid_result = grid.fit(X, y, ) #callbacks=[tbCallBack]

# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored